In [ ]:
!python main.py

In [ ]:
# Master reset all databases and posts
from main import reset

reset()

In [ ]:
# Only remake all .md posts
from scholar.post_maker import make_all

make_all(overwrite=True)

# Archive

## Add new column called arxiv_term in SQLite

In [ ]:
# import sqlite3

# Reset the new columns
# with sqlite3.connect('scholar/paper.db') as conn:
#     c = conn.cursor()
#     c.execute('ALTER TABLE paper DROP COLUMN arxiv_term_short;')
#     c.execute('ALTER TABLE paper DROP COLUMN arxiv_term_long;')
#     conn.commit()

# with sqlite3.connect("scholar/paper.db") as conn:
#     c = conn.cursor()
#     c.execute("ALTER TABLE paper ADD COLUMN arxiv_group_tag VARCHAR(255);")
#     c.execute("ALTER TABLE paper ADD COLUMN arxiv_category_tag VARCHAR(255);")
#     conn.commit()

## Patch category and group tag to database

In [ ]:
import arxiv

search = arxiv.Search(query=f"ti:Simulation-based inference", max_results=1)
result = next(search.results())

In [ ]:
result.entry_id.split("/")[-1]

In [ ]:
def arxiv_to_bibtex(entry):
    bibtex = "@article{{{},\n".format(entry.get("id").replace("/", "_"))

    # Title
    title = entry.get("title").replace("\n", " ").replace("\r", "").strip()
    bibtex += "  title={{ {} }},\n".format(title)

    # Author(s)
    authors = " and ".join(author.name for author in entry.get("authors"))
    bibtex += "  author={{ {} }},\n".format(authors)

    # Journal
    bibtex += "  journal={{ arXiv preprint }},\n"

    # Year
    year = entry.get("published")[:4]
    bibtex += "  year={{ {} }},\n".format(year)

    # URL
    url = entry.get("id")
    bibtex += "  url={{ {} }},\n".format(url)

    # eprint (arXiv ID)
    eprint = re.sub(r"http[s]?://arxiv.org/abs/", "", entry.get("id"))
    bibtex += "  eprint={{ {} }}\n".format(eprint)

    bibtex += "}\n"

    return bibtex

In [ ]:
arxiv_to_bibtex(result)